<a href="https://colab.research.google.com/github/gmseabra/MolDB-Exploration/blob/main/ChEMBL_COX-2_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if 'google.colab' in sys.modules:
    print('Running on CoLab - Installing packages')
    !pip install rdkit pandas py3Dmol mols2grid matplotlib seaborn tqdm scikit-learn
    !mkdir data
    !wget https://github.com/gmseabra/MolDB-Exploration/blob/52ea789c55ba89377e37d75211ebfbd08f631e9b/data/ChEMBL_COX-2_2024-08-28.csv -O data/ChEMBL_COX-2_2024-08-28.csv
    !mkdir utils
    !wget https://github.com/gmseabra/MolDB-Exploration/blob/52ea789c55ba89377e37d75211ebfbd08f631e9b/utils/preprocess.py -O utils/utils.py
else:
    print('Not running on CoLab - Assuming packages are already installed')

Running on CoLab - Installing packages
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.0 MB/s eta 0:00:00
--2025-08-25 22:33:08--  https://github.com/gmseabra/MolDB-Exploration/blob/52ea789c55ba89377e37d75211ebfbd08f631e9b/data/ChEMBL_COX-2_2024-08-28.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/ChEMBL_COX-2_2024-08-28.csv’

data/ChEMBL_COX-2_2     [ <=>                ] 181.11K  --.-KB/s    in 0.009s  

2025-08-25 22:33:08 (18.9 MB/s) - ‘data/ChEMBL_COX-2_2024-08-28.csv’ saved [185456]

--2025-08-25 22:33:08--  https://github.com/gmseabra/MolDB-Exploration/blob/52ea789c55ba89377e37d75211ebfbd08f631e9b/utils/preprocess.py
Resolving github.com (gith

# Imports

In [ ]:
# -- Data & Plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Progress Bar
from tqdm import tqdm, tnrange, tqdm_notebook
tqdm.pandas()

# -- RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
PandasTools.RenderImagesInAllDataFrames()
print("RDKit Version: ", rdkit.__version__)

In [ ]:
# This will allow us to see all columns:
pd.set_option("display.max_columns", 100)

# To disable it, do:
# pd.reset_option("^display")

# To get details:
pd.get_option("display.max_columns")

# More information at:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html#getting-and-setting-options

# Obtaining the database
Today we will be working with a database of compounds that wre somehow tested against the
human cycloxoygenase-2 ([COX-2](https://en.wikipedia.org/wiki/Cyclooxygenase-2)),
which is involved in pain and inflammation processes.
NSAIDS that selectively target COX-2 ([coxibs](https://en.wikipedia.org/wiki/Cyclooxygenase-2_inhibitor))
reduce the risk of peptic ulcers associated with the inhibition of a related enzyme,
[COX-1](https://en.wikipedia.org/wiki/Cyclooxygenase-1).

We can obtain a database of compounds tested against COX-2 by searching different databases,
the main ones being [PubChem](https://pubchem.ncbi.nlm.nih.gov/), maintained by the NIH,
and [ChEMBL](https://www.ebi.ac.uk/chembl/), maintained by the European Bioinformatics Institute.

Here, we will work with compounds obtained from the ChEMBL database. The process to obtain the database is the following:
1. Navigate to the [ChEMBL](https://www.ebi.ac.uk/chembl/) site, and search for "COX-2", **but don't hit enter yet!**
2. From the drop-down list, under "Targets", select "COX-2". That will return a page with different "COX-2" targets
3. Scroll down to the human COX-2 (CHEMBL230):
<a href="https://www.ebi.ac.uk/chembl/target_report_card/CHEMBL230/"><img src="https://github.com/gmseabra/MolDB-Exploration/blob/main/media/ChEMBL_COX-2.png?raw=1" border=1 /></a> <br>
Clicking on the header will take you to the page with COX-2 data.
4. Finally,scroll to **"Activity Charts"**, and click on the **"Activity Types for CHEMBL230 (Cyclooxygenase-2)"** link:
<a href="https://www.ebi.ac.uk/chembl/web_components/explore/activities/STATE_ID:A7tQ9Eh8n8uuiWNfvBq-yw%3D%3D"><img src="https://github.com/gmseabra/MolDB-Exploration/blob/main/media/ChEMBL_COX-2_activities.png?raw=1" border=1 /></a> <br>
It will take you to a list of 13,999 compounds (as of 08/28/2024) with measured activity against COX-2,
which you can download as a CSV file by clicking the <img src="https://github.com/gmseabra/MolDB-Exploration/blob/main/media/ChEMBL_CSV.png?raw=1" alt="CSV" /> button.


<div class="alert alert-info" role="alert">
    Note: It may take a while. To speed up our class, the file is already available here in the data folder: `data/ChEMBL_COX-2_2024-08-28.csv`.
</div>


And we're ready to start exploring!

# Exploring the database
The CSV file from ChEMBL has a weird format, with ";" separating the columnns!

In [ ]:
cox2db = pd.read_csv("data/ChEMBL_COX-2_2024-08-28.csv", sep=";")
cox2db.head(5)

That's a lot of stuff. This is the list of columns available:

In [ ]:
cox2db.columns

In [ ]:
cox2db.shape

## Unique molecules

The database has a total of 13,999 *entries*, which were collected from diverse sources. How many are really unique? We can check by the `ChEMBL ID`, which is an unique identifier for each molecule:

In [ ]:
len(cox2db['Molecule ChEMBL ID'].unique())

From the total of 13,999 entries, only 8,582 have unique IDs.

Furthermore, if we check the SMILES representations:

In [ ]:
len(cox2db['Smiles'].unique())

There seems to be some molecules with different IDs but the same SMILES!

Let's look at the repetitions of molecules with the same ID:

In [ ]:
id_counts = cox2db['Molecule ChEMBL ID'].value_counts()
id_counts

Some molecules appear in various experiments. For example, Celecoxib ([CHEMBL118](https://www.ebi.ac.uk/chembl/compound_report_card/CHEMBL118/)) is a reference in many studies.

In [ ]:
Chem.MolFromSmiles(cox2db[ cox2db['Molecule ChEMBL ID'] == 'CHEMBL118']['Smiles'].unique()[0])

## Example: Celecoxib (CHEMBL118)

It is likely that the repetitions mean different assays. Especially, some common known COX-2 inhibitors are used in many assays as reference. For example, Celecoxib ([CHEMBL118](https://www.ebi.ac.uk/chembl/compound_report_card/CHEMBL118/)) appears in 287 different assays of 10 different types:

In [ ]:
len(cox2db[ cox2db['Molecule ChEMBL ID'] == 'CHEMBL118']['Assay ChEMBL ID'].unique())

In [ ]:
cox2db[ cox2db['Molecule ChEMBL ID'] == 'CHEMBL118']['Standard Type'].value_counts()

### $IC_{50}s$

Most values (209) are for IC50, which is notoriously imprecise. Note the variation in values:

In [ ]:
cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') & (cox2db['Standard Type'] == 'IC50')]["Standard Value"].describe()

First, let's check how many different units are used:

In [ ]:
cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') & (cox2db['Standard Type'] == 'IC50')]["Standard Units"].unique()

Good... At least it seems to have only one unit reported. So, let's take a look into the results:

In [ ]:
g = sns.stripplot(data = cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') &
                                 (cox2db['Standard Type'] == 'IC50')
                               ],
                   x = "Standard Value")
g.set(xlabel='IC50 Value')

Ugh!!

The most likely reason for the very high values are wrong reported units or typos. ChEMBL provides a column with comments about data validity:

In [ ]:
cox2db['Data Validity Comment'].unique()

Let's see which comments are in the database:

In [ ]:
cox2db['Data Validity Comment'].value_counts()

The `Data Validity Comment` column only has any contents when there is some indication that the data may be wrong. So, we can use this column to filter out obviuos outliers and dubious data. Does it get better?

In [ ]:
g = sns.stripplot(data = cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') &
                                 (cox2db['Standard Type'] == 'IC50') &
                                 (cox2db['Data Validity Comment'].isna())
                               ],
                   x = "Standard Value")
g.set(xlabel='IC50 Value')

A bit better... But there's still some values that are way too high. If we remove the aberrant values above 1,000, we get:

In [ ]:
g = sns.swarmplot(data = cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') &
                                 (cox2db['Standard Type'] == 'IC50') &
                                 (cox2db['Data Validity Comment'].isna()) &
                                 (cox2db['Standard Value'] < 1000)
                               ],
                   x = "Standard Value")
g.set(xlabel='IC50 Value')

So, what is the $IC_{50}$ for Celecoxib, after all?

As an aside, instead of the $IC_{50}s$, we can use the $-Log(IC_{50})$ value, which has less variation. Let's look at it's distribution:

In [ ]:
sns.swarmplot(data=cox2db[(cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') &
                          (cox2db['Standard Type'] == 'IC50') &
                          (cox2db['Standard Relation'] == "'='") &
                          (cox2db['Data Validity Comment'].isna()) &
                          (cox2db['Standard Value'] < 1000)
                        ],
             x='pChEMBL Value')

### Inhibition

66 of the experiments report "Inhibition":

In [ ]:
# Inhibition
data = cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') &
               (cox2db['Standard Type'] == 'Inhibition')     &
               (cox2db['Standard Units'] == '%')]

sns.histplot(data = data, x='Standard Value')

In [ ]:
data["Standard Value"].describe()

### Activity

8 entries are just labeled "Activity", but the information seems meaningless. They are probably relative to some other compound, not shown here. That means those numbers cannot be used!

In [ ]:
data = cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') & (cox2db['Standard Type'] == 'Activity')]
sns.barplot(data=data, x=data.index, y='Standard Value')

### $\Delta G$, $K_a$ and $K_i$

Those are usually the most precise types of data, but are harder to get. There's only 4 values in the data, and it turns out the numbers in the database are not very informative:

In [ ]:
data = cox2db[ (cox2db['Molecule ChEMBL ID'] == 'CHEMBL118') &
                 ((cox2db['Standard Type'] == 'Delta G') |
                  (cox2db['Standard Type'] == 'Ka')      |
                  (cox2db['Standard Type'] == 'Ki')
                 )]
data

This $K_a$ can be converted to a $\Delta G$ by

$\Delta G = -R T Ln(K_a)$.

Using:
- $R = 1.987 \times 10^{-3} kcal \cdot K^{−1} \cdot mol−1$
- $T = 298 K$

we get $\Delta G = -5.91 kcal/mol$ .

## Exploring the Data Types in the Full Database

In [ ]:
data = cox2db['Standard Type'].value_counts()
data

In [ ]:
g = sns.barplot(x=data.index, y=data.values)
g = g.set_xticklabels(g.get_xticklabels(),rotation=90)

So, most of the data is in the form of IC50s. Some molecules also have inhibition, Ki and activity. Let's take a look at the different kinds of data.

### $IC_{50}s$

We got lucky with Celecoxib. When we look at the full data, the IC50 data is reported in many different units:

In [ ]:
data = cox2db[cox2db['Standard Type'] == 'IC50']
data['Standard Units'].value_counts()

We can probably discard all that are not in nM units. Still, even if we only consider the data in 'nM', apparently there's data without values, either in 'Standard Value' or 'pChEMBL value':

In [ ]:
data = data[ data['Standard Units'] == 'nM' ]
data['Standard Value'].isna().any(), data['pChEMBL Value'].isna().any()

19 entries are missing the values:

In [ ]:
data['Standard Value'].isna().sum()

Also, 1 of them don't have SMILES string:

In [ ]:
data['Smiles'].isna().sum()

Some were out of detection limits:

In [ ]:
data['Standard Relation'].value_counts()

The maximum and minimum values are also weird an $IC_{50}$ of 0 nM makes no sense:

In [ ]:
data['Standard Value'].max(), data['Standard Value'].min()

In [ ]:
data['Standard Value'].describe()

when preparing a final database, we'll need to filter those outliers.

### Inhibition

In [ ]:
data = cox2db[ cox2db['Standard Type'] == 'Inhibition' ]

Some inhibitions are in $\mu M$ units, which doesn't make much sense. Let's remove those.

In [ ]:
data['Standard Units'].value_counts()

In [ ]:
data = data[ data['Standard Units'] == '%' ]

Some of the values are empty:

In [ ]:
data['Standard Value'].isna().any()

In [ ]:
data['Standard Value'].describe()

In [ ]:
data['Standard Value'].hist()

What does it mean to have **negative** % inhibition????

In [ ]:
data[data['Standard Value'] < 0].head()

A look at the source for the second entry reveals that this compound can actually be considered an activator of COX-2!!
 - ChEMBL Document [CHEMBL1149153](https://www.ebi.ac.uk/chembl/document_report_card/CHEMBL1149153/)
 - J. Med. Chem. 2004, 47, 9, 2180–2193 [here](https://pubs.acs.org/doi/10.1021/jm030276s)


That means that, to use inhibitions, we should also remove data vith inhibition < 0.

**Exercise:** Check the presence of valid dat and SMILES.

### $AC_{50}s$

The third in the list is the concentration for half-maximal activity, $AC_{50}$:

In [ ]:
data = cox2db[cox2db['Standard Type'] == 'AC50']
data['Standard Units'].value_counts()

So, they are all in the same units, which is good. However, some are missing the 'pChEMBL value':

In [ ]:
data['Standard Value'].isna().any(), data['pChEMBL Value'].isna().any()

There's also a weird variation in values:

In [ ]:
data['Standard Value'].max(), data['Standard Value'].min()

In [ ]:
data['Standard Value'].describe()

In [ ]:
sns.histplot(data = data, x='Standard Value')

Again, Ugh! The very large values are likely "inactive" molecules, where the data is reported just as _greater than_ some detection limit $X$

In [ ]:
data["Standard Relation"].value_counts()

So, only about 141 points with real numbers.

### $K_i$

Next, lets look at $K_i$, with 899 entries.

In [ ]:
data = cox2db[ cox2db['Standard Type'] == 'Ki' ]

From those, most have no value reported, because the dose-response curve could not be determined:

In [ ]:
data['Standard Value'].isna().sum()

In [ ]:
data.head(1)

In [ ]:
print(data.loc[0]['Comment'])

Ant those that have a value reported include a range of different data. Some of the data are only " > " a certain threshold, indicating not active.

In [ ]:
data[data['Standard Value'].notna()]['Standard Relation'].value_counts()

Also, some have $K_i = 0$:

### Activity
Activity, which is listed for 398 cases:

In [ ]:
data = cox2db[ cox2db['Standard Type'] == 'Activity' ]

In [ ]:
data['Standard Units'].value_counts()

In [ ]:
data[ data['Standard Units'] == 'ng/ml' ].head()

This is a relly hard data to analyze. Looking at one document [here](https://pubs.acs.org/doi/10.1021/acs.jmedchem.8b00922), we see that the data listed as "inhibition" actually is the . _"Inhibition of TxB2 during Calcium-Ionophore Stimulation and Inhibition of PGE2 during LPS-Stimulation of Human Whole Blood"_, at the compound concentration of 1 μM.

The only way to use this data is by looking at each article case-by-case! However, the same papers can also have IC50s for the molecules, so we can use that instead.

# Build Database

There are a total of 8,545 compounds wth unique SMILES in the database. However, the database actually has:

- 7891 entries for $IC_{50}$ (which seems to be the most complete type of data).
- 3315 entries with "Inhibition" data
- 1193 with "$AC_{50}$s"
- 899 with $K_i$ data, from which only 25 acctually have any data to it.
- 398 with "Activity" data.

Let's create a database with only useful data. It is likely that we can get *some* data on all molecules by using a combination of these.

## Columns
There's a large number of columns, with all kinds of data in the database:

In [ ]:
cox2db.columns

That's a lot of columns, and not all are necessary. Let's keep only a selection of these:

In [ ]:
columns = ['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
           'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value','Standard Units']

## IC50s

Let's do some checking first. We start with the 7,891 entries with IC50 data.

**Remember that:**
- Some are reported in different units,
- some have values or SMILES missing,
- Some are only annotated as _"larger than"_ or _"smaller than"_ some threshold
- Some have `Data Validity Comment`s indicating the data may be problematic

We will need to remove all those cases.

Lets start only with data for which we have:
1. Measured IC50 values in units of nM
1. There is a Smiles value
1. Have no warnings on data value

In [ ]:
ic50_data = cox2db[ (cox2db['Standard Type']     == 'IC50')  &
                    (cox2db['Standard Relation'] ==  "'='")  &
                    (cox2db['Standard Units']    ==  'nM' )  &
                    (cox2db['Smiles'].notna()             )  &
                    (cox2db['Data Validity Comment'].isna()) ][columns]

print("Total number of entries that match the criteria:", len(ic50_data))
print("Number of entries with unique SMILES strings:   ", len(ic50_data['Smiles'].unique()))
print("Do the number of unique SMILES match the number of unique ChEMBL IDs?",
      len(ic50_data['Smiles'].unique()) == len(ic50_data['Molecule ChEMBL ID'].unique()))

### Wisdom of the crowd

There are a total of 5,571 entries that match the criteria, but only 4,011 unique molecules. That means there a lot of molecules appear more than once (remember Celecoxib?). If the values vary so much, how do we choose the correct one?

We *could* just average the values, but that may be skewed by outliers. Here, we will use the *Wisdom of the Crowd* idea: for each molecule, we will choose the **most common** value (`mode`).

In [ ]:
# This creates a new DataFrame to hold unique entries, and use the `mode` of IC50s for that molecule as the value.
cox2_ic50s = pd.DataFrame(columns=columns)
for  ic50_data_line in tqdm(ic50_data['Molecule ChEMBL ID'].unique()):
    new_row =  ic50_data[ ic50_data['Molecule ChEMBL ID'] ==  ic50_data_line].mode().head(1).copy(deep=True)
    cox2_ic50s = pd.concat([cox2_ic50s,new_row])
cox2_ic50s.shape

In [ ]:
cox2_ic50s.head()

In [ ]:
print("The new database has shape: ", cox2_ic50s.shape)
print("The new database has length: ", len( cox2_ic50s['Smiles'].unique() ))
print("Number of unique ChEMBL IDs: ", len( cox2_ic50s['Molecule ChEMBL ID'].unique() ))

Finally, let's remove the temporary dataframes, to save memory:

In [ ]:
del(ic50_data, ic50_data_line)

### Activity Bit
The numerical data we have so far is good for regression. We can also add a column to represent
if the molecule is active or not, useful for classification purposes. Here, let's define a molecule
as *acctive* if the $IC_{50}$ is below 100 nM:

In [ ]:
cox2_ic50s["Active"] = cox2_ic50s["Standard Value"] < 100

In [ ]:
cox2_ic50s["Active"].value_counts()

We have about 3 inactives for each active compound.

### Other Relationships

We can now go back to the other relationships, Furst, look at the molecules with a "larger than" relationship. Those were so inactive they couldn't even measure the $IC_{50}$s!

In [ ]:
ic50_lt_data = cox2db[ (cox2db['Standard Type']     == 'IC50')  &
                       (cox2db['Standard Relation'] ==  "'>'")  &
                       (cox2db['Smiles'].notna()             )  &
                       (cox2db['Data Validity Comment'].isna()) ][columns]

However, not all are unique, and some were already counted before:

In [ ]:
cox2_ic50s_mols = set(cox2_ic50s['Molecule ChEMBL ID'].unique())

lt_mols = set(ic50_lt_data['Molecule ChEMBL ID'].unique())
new_mols = lt_mols - cox2_ic50s_mols
print("Total number of molecules with 'lt' relationship data:", len(ic50_lt_data))
print("Number of unique molecules with 'lt' relationship data: ", len(lt_mols) )
print("Number of those without IC50 data:", len(new_mols))

Still, we can sue the data for 811 new molecules. Let's just set them all as inactive, and marge to the previous database.

In [ ]:
ic50_lt_data.drop_duplicates(subset=['Molecule ChEMBL ID'], inplace=True)

In [ ]:
for mol in tqdm(ic50_lt_data['Molecule ChEMBL ID']):
    if (mol in cox2_ic50s_mols):
        ic50_lt_data.drop( ic50_lt_data.loc[ic50_lt_data['Molecule ChEMBL ID'] == mol].index, inplace=True)

In [ ]:
len(ic50_lt_data)

Let's now set the `Active` bit of those to "False" (inactive) and merge to the database:

In [ ]:
ic50_lt_data['Active'] = False
cox2_ic50s = pd.concat([cox2_ic50s,ic50_lt_data], ignore_index=True)
print(f"The IC50s database now has {len(cox2_ic50s)} unique molecules.")

### $IC_{50}$s Summary

From an initial total of 13,999 entries, there were only 8,545 unique molecules. From those, only  4,011 were unique molecules with reasonably useful IC50 data.

### A quick peek at the molecules

In [ ]:
# A quick look at the molecules
n_to_draw = 8
df = cox2_ic50s.sort_values(by=["Standard Value"],ascending=[True]).head(n_to_draw)
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='Smiles')
mols = df.ROMol.values
top_ic50s = df['Molecule ChEMBL ID'].values
legends = []
for chemblid, ic50, unit in zip(df['Molecule ChEMBL ID'].values, df['Standard Value'].values, df['Standard Units']):
    legends.append(f"{chemblid}\nIC50={ic50:.3f}{unit}")
Draw.MolsToGridImage(mols, molsPerRow=4,subImgSize=(250,250), legends=legends)

## Other Properties

There are also some molecules with only other types of activity reported:

In [ ]:
other_properties = ['Inhibition', 'AC50','Ki','Activity']

cox2_ic50s_mols = set(cox2_ic50s['Molecule ChEMBL ID'].unique())
for prop in other_properties:
    prop_data = cox2db[ (cox2db['Standard Type']     == prop)  &
                        (cox2db['Standard Relation'] ==  "'='")  &
                        (cox2db['Smiles'].notna()             )  &
                        (cox2db['Data Validity Comment'].isna()) ][columns]
    print(f"Number of entries with data as {prop}: {len(prop_data)}")
    prop_mols = set(prop_data['Molecule ChEMBL ID'].unique())
    new_mols = prop_mols - cox2_ic50s_mols
    print("Number of those without IC50 data:", len(new_mols))
    print("-"*60)

It looks like we can recover some more data using those measures.

<div class="alert alert-info" role="alert">
    <b>Exercise:</b> Try to gather more data from those other measures and enrich your database.
</div>

## Pre-process database

Now we need to pre-process the database, to:

1. Remove molecules that contain atoms other than ['C','N','O','H','S','P','As','Se','F','Cl','Br','I']
1. Standardize (remove salts, etc.)
1. Reset the SMILES from the ROMol objects
1. Remove duplicates

This process involves a number of functions. To make it easier, we collect all those functions
in an external file (`utils/preprocess.py`), and just import it here.

<div class="alert alert-warning" role="alert">
    <b>Question:</b> Why didn't we do this earlier?
</div>

In [ ]:
# -- Preprocessing Tools
from utils import preprocess

In [ ]:
clean_df, dup_df = preprocess.preprocess_db(cox2_ic50s, smiles_col='Smiles')

So, the there were 52 molecules with atoms other than ['C','N','O','H','S','P','As','Se','F','Cl','Br','I']. Other than that, all was fine.

In [ ]:
cox2_ic50s.shape, clean_df.shape, dup_df.shape

In [ ]:
clean_df.columns

In [ ]:
# A quick look at the molecules
n_to_draw = 8
df = clean_df[ clean_df['Molecule ChEMBL ID'].isin(top_ic50s) ]
mols = df.ROMol.values
legends = []
for chemblid, ic50, unit in zip(df['Molecule ChEMBL ID'].values, df['Standard Value'].values, df['Standard Units'].values):
    legends.append(f"{chemblid}\nIC50={ic50:.3f} {unit}")
Draw.MolsToGridImage(mols, molsPerRow=4,subImgSize=(250,250), legends=legends)

In [ ]:
cox2_ic50s = clean_df.copy(deep=True)

We no longer need these other databases, let's remove them.

In [ ]:
del(clean_df,dup_df)

In [ ]:
cox2_ic50s.sample(5)

In [ ]:
len(cox2_ic50s)

In [ ]:
len(cox2_ic50s['Molecule ChEMBL ID'].unique())

## Save database

# Example: A Random Forest Classifier

As a usage example, let's build a simple Random Forest Classifier to determine if a molecule is active against COX-2 or not.

## Featurize the Molecules
To create a model, we will need to add features to the molecules. Here, let's use the molecular fingerprint as features.

In [ ]:
fpgen = AllChem.GetMorganGenerator(radius=3,fpSize=2048)

In [ ]:
cox2_ic50s['MorganFP'] = [fpgen.GetFingerprintAsNumPy(x) for x in cox2_ic50s.ROMol]

In [ ]:
cox2_ic50s.head()

## Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, y = cox2_ic50s.MorganFP.values.tolist(), cox2_ic50s.Active.values.tolist()

In [ ]:
len(X), len(y)

Let's use 80% of the molecules for training, and test the model with the remaining 20%:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.30,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [ ]:
print('Training set: ', len(X_train))
print('Testing set:  ', len(X_test))

In [ ]:
values,counts = np.unique(y_train, return_counts=True)
for value, count in zip(values, counts):
    print(f"Active? {value!s:5}: {count: 6d}")
print(f"Proportion of inactive molecules in the training set: {counts[0]/(counts[0] + counts[1]):0.2f}")

In [ ]:
values,counts = np.unique(y_test, return_counts=True)
for value, count in zip(values, counts):
    print(f"Active? {value!s:5}: {count: 6d}")
print(f"Proportion of inactive molecules in the testing set: {counts[0]/(counts[0] + counts[1]):0.2%}")

## Train the model

For this example, we will use a simple Random Forest. This is always a good baseline: Any model you create should be at least as good as a Random Forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(random_state=42)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

## Test the Model

In [ ]:
from sklearn import metrics

In [ ]:
print(f"Accuracy:  {metrics.accuracy_score(y_test, y_pred):.2f}")
print(f"F1-Score:  {metrics.f1_score(y_test, y_pred):.2f}")
print(f"Precision: {metrics.precision_score(y_test, y_pred):.2f}")
print(f"Recall:    {metrics.recall_score(y_test, y_pred):.2f}" )
print(f"ROC_AUC:   {metrics.roc_auc_score(y_test, y_pred):.2f}" )

Not a bad model, huh? Especially considering there we did not optimize any hyperparamters!

<div class="alert alert-success" role="alert">
    <b>That's it for today</b> </br>
    As an exercise, see if you can get a better model by adding more data from the other measures or adjusting hyperparameters!
</div>